In [87]:
import pandas as pd
import pylab
import matplotlib.pyplot as plt
import time
from pathlib import Path
from sklearn import preprocessing
import numpy as np
import os

In [88]:
parent_path = "./AETA/training/"
num_blocks = 3
_ranges = [[23.0, 26.526666666666667, 98.43, 101.0], [23.0, 26.526666666666667, 101.0, 103.57], [23.0, 26.526666666666667, 103.57, 106.13999999999999], [26.526666666666667, 30.053333333333335, 98.43, 101.0], [26.526666666666667, 30.053333333333335, 101.0, 103.57], [26.526666666666667, 30.053333333333335, 103.57, 106.13999999999999], [30.053333333333335, 33.58, 98.43, 101.0], [30.053333333333335, 33.58, 101.0, 103.57], [30.053333333333335, 33.58, 103.57, 106.13999999999999]]

In [89]:
station_info = pd.read_csv("StationInfo.csv")
stations = list(station_info['StationID'])
station_lati = list(station_info['Latitude'])
station_long = list(station_info['Longitude'])

In [90]:
eq_info = pd.read_csv("eqlst.csv")

In [91]:
min_stn_lati = min(station_lati)
min_stn_long = min(station_long)
max_stn_lati = max(station_lati)
max_stn_long = max(station_long)
latitude_chunk = (max_stn_lati - min_stn_lati) / num_blocks
longitude_chunk = (max_stn_long - min_stn_long) / num_blocks

min_eq_lati = min(eq_info["Latitude"])
max_eq_lati = max(eq_info["Latitude"])
min_eq_long = min(eq_info["Longitude"])
max_eq_long = max(eq_info["Longitude"])

eq_coor = list(zip(eq_info["Longitude"], eq_info["Latitude"]))

In [92]:
_ranges

[[23.0, 26.526666666666667, 98.43, 101.0],
 [23.0, 26.526666666666667, 101.0, 103.57],
 [23.0, 26.526666666666667, 103.57, 106.13999999999999],
 [26.526666666666667, 30.053333333333335, 98.43, 101.0],
 [26.526666666666667, 30.053333333333335, 101.0, 103.57],
 [26.526666666666667, 30.053333333333335, 103.57, 106.13999999999999],
 [30.053333333333335, 33.58, 98.43, 101.0],
 [30.053333333333335, 33.58, 101.0, 103.57],
 [30.053333333333335, 33.58, 103.57, 106.13999999999999]]

In [93]:
def xInRange(x, block_index):
    '''
    x: longtitude
    '''
    _min = _ranges[block_index][2]
    _max = _ranges[block_index][3]

    if block_index % 3 == 0:
        return x < _min or (x >= _min and x < _max)
    elif block_index % 3 == 1:
        return x >= _min and x < _max
    else:
        return x > _max or (x >= _min and x < _max)

def yInRange(y, block_index):
    '''
    y: latitude
    '''
    _min = _ranges[block_index][0]
    _max = _ranges[block_index][1]

    if block_index < num_blocks:
        return y < _min or (y >= _min and y < _max)
    elif block_index < num_blocks*2:
        return y >= _min and y < _max
    else:
        return y > _max or (y >= _min and y < _max)


    
def inblock(x, y, block_index):
    '''
    x, y: 地震的long，lati
    block_index: block在_ranges里的index
    '''
    return xInRange(x, block_index) and yInRange(y, block_index)

In [95]:
# x = long, y = lati
blocks = []
for x, y in eq_coor:
    for i in range(num_blocks * num_blocks):
        if inblock(x, y, i):
            blocks.append(i)
            break
assert(len(blocks) == len(eq_coor))
eq_info['Block'] = blocks
eq_info.to_csv("tagged_eq_info.csv", index = False, header=True)

In [86]:
from sklearn.cluster import KMeans

# 有一个问题，kmeans是随机的，每次结果都不一样
kmeans = KMeans(n_clusters=8)
kmeans.fit(list(zip(station_long, station_lati)))
y_kmeans = kmeans.predict(eq_coor)

center = kmeans.cluster_centers_
print(center)
assert(len(y_kmeans) == len(eq_coor))

for i in range(len(y_kmeans)):
    print(y_kmeans[i], eq_coor[i])

[[100.07555556  31.04      ]
 [102.482       24.116     ]
 [104.54116668  32.74631112]
 [102.23232544  28.44335039]
 [100.37040326  27.43335968]
 [103.83761905  28.93785714]
 [ 99.55769231  25.51384615]
 [102.98744083  31.24425861]]
3 (101.39, 27.92)
5 (105.6, 29.47)
5 (105.55, 29.43)
5 (104.72, 28.16)
5 (104.73, 28.14)
5 (104.76, 28.1)
5 (103.61, 30.1)
5 (104.72, 28.09)
5 (104.7, 28.15)
3 (103.36, 27.07)
7 (103.38, 31.02)
3 (103.4, 27.09)
6 (99.83, 25.87)
6 (99.8, 25.89)
6 (99.81, 25.85)
3 (103.36, 27.11)
2 (104.56, 32.21)
7 (103.97, 31.44)
5 (104.87, 28.19)
6 (99.34, 25.65)
7 (103.18, 31.31)
5 (104.67, 28.16)
1 (101.71, 25.12)
2 (105.36, 32.38)
2 (103.82, 33.2)
2 (103.76, 33.3)
2 (103.87, 33.12)
2 (103.87, 33.12)
2 (103.8, 33.28)
2 (103.85, 33.16)
2 (103.75, 33.28)
2 (103.86, 33.16)
2 (103.79, 33.22)
2 (103.85, 33.16)
2 (103.85, 33.22)
2 (103.87, 33.12)
5 (105.42, 28.93)
3 (101.42, 27.93)
2 (105.0, 32.27)
2 (103.8, 33.23)
2 (105.52, 32.51)
7 (103.7, 31.22)
2 (103.79, 33.21)
7 (103.97